# Setting up

In [1]:
import os
import pandas as pd
import numpy as np
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import re
import nltk
import pymorphy2
from tqdm import tqdm

In [2]:
files = os.listdir('IT')

In [3]:
vacancies_it = pd.DataFrame()

for file in files:
    sub_category_name = file.split('.csv')[0]
    
    sub_category_df = pd.read_csv('IT/' +file, index_col=0)
    
    sub_category_df['SUBCATEGORY'] = sub_category_name
    
    vacancies_it = pd.concat([vacancies_it, sub_category_df], ignore_index=True)
    

In [4]:
vacancies_it.head()

,Название,Компания,Местоположение,Занятость,Требования по умениям,Требования по опыту,Должностные обязанности,Зарплата,Ключевые навыки,SUBCATEGORY
0,Программист-разработчик (SQL),ООО Финансово-правовой альянс,"Сенная площадь, Санкт-Петербург, улица Ефимова...","Полная занятость, полный день",NaN,3–6 лет,"Разработка и внедрение ПО, охватывающего весь...",до 300 000 руб. на руки,"Базы данных, Анализ данных, SQL, Разработка ПО...",системы управления предприятием (erp)
1,Программист C#/ .NET,ООО Syntellect (Синтеллект),Санкт-Петербург,"Полная занятость, удаленная работа",Опыт работы с WPF Опыт в автоматизации задач ...,1–3 года,Высшее\неполное высшее образование в сфере ИТ...,от 60 000 до 150 000 руб. до вычета налогов,"C#, Transact-SQL, SQL, MS SQL, .NET Framework,...",системы управления предприятием (erp)
2,Системный и Бизнес-аналитик,Fortis,"Звёздная, Санкт-Петербург, Звёздная улица, 1","Полная занятость, полный день",NaN,1–3 года,,з/п не указана,,системы управления предприятием (erp)
3,Руководитель проекта (разработка и внедрение ПО),Наука,"Политехническая, Санкт-Петербург, Гжатская ули...","Полная занятость, полный день",NaN,3–6 лет,,от 90 000 руб. на руки,"MS Project, Управление проектами, Управление р...",системы управления предприятием (erp)
4,Java Team lead,Наука,"Политехническая, Санкт-Петербург, Гжатская ули...","Полная занятость, полный день",NaN,не требуется,,з/п не указана,,системы управления предприятием (erp)


# Preparing vacancies data set

In [5]:
vacancies_it['Ключевые навыки'] = vacancies_it['Ключевые навыки'].apply(lambda x: re.sub(',', '.', x))

In [6]:
vacancies_it = vacancies_it.fillna(' ')

In [7]:
vacancies_it_with_rs = vacancies_it.copy()

In [8]:
vacancies_it_with_rs['REQ+SKILLS'] = vacancies_it['Ключевые навыки'] + ' ' + vacancies_it['Требования по умениям']

In [9]:
vacancies_it_with_rs['REQ+SKILLS'] = vacancies_it_with_rs['REQ+SKILLS'].apply(lambda x : re.sub(r'\s+', ' ', x))

In [10]:
vacancies_it_with_rs = vacancies_it_with_rs[vacancies_it_with_rs['REQ+SKILLS'] != ' ']

In [11]:
vacancies_it_with_rs = vacancies_it_with_rs.drop(['Должностные обязанности', 'Ключевые навыки', 'Требования по умениям'], axis=1)

In [12]:
vacancies_it_with_rs.head()

,Название,Компания,Местоположение,Занятость,Требования по опыту,Зарплата,SUBCATEGORY,REQ+SKILLS
0,Программист-разработчик (SQL),ООО Финансово-правовой альянс,"Сенная площадь, Санкт-Петербург, улица Ефимова...","Полная занятость, полный день",3–6 лет,до 300 000 руб. на руки,системы управления предприятием (erp),Базы данных. Анализ данных. SQL. Разработка ПО...
1,Программист C#/ .NET,ООО Syntellect (Синтеллект),Санкт-Петербург,"Полная занятость, удаленная работа",1–3 года,от 60 000 до 150 000 руб. до вычета налогов,системы управления предприятием (erp),C#. Transact-SQL. SQL. MS SQL. .NET Framework....
3,Руководитель проекта (разработка и внедрение ПО),Наука,"Политехническая, Санкт-Петербург, Гжатская ули...","Полная занятость, полный день",3–6 лет,от 90 000 руб. на руки,системы управления предприятием (erp),MS Project. Управление проектами. Управление р...
5,Программист 1С,Неркон,"Санкт-Петербург, Площадь Ленина, Выборгская","Полная занятость, удаленная работа",более 6 лет,от 25 000 до 45 000 руб. на руки,системы управления предприятием (erp),1С программирование. 1С: Управление Торговлей....
6,Программист PHP Битрикс,ООО Вебдока,"Площадь Ленина, Санкт-Петербург, Кондратьевски...","Полная занятость, удаленная работа",3–6 лет,от 60 000 до 110 000 руб. на руки,системы управления предприятием (erp),битрикс. bitrix. yii. symfony


In [13]:
re.split(r'[.;]', vacancies_it_with_rs['REQ+SKILLS'].iloc[5])

['ERP-системы на базе 1С',
 ' ERP Опыт внедрения 1С: ERP на производственном предприятии обязателен',
 ' Опыт внедрения 1С:ERP на предприятии с количеством пользователей от 20 человек обязательно',
 ' Сертификат, подтверждающий квалификацию от 1С',
 ' Рекомендации и(или) портфолио успешных проектов внедрения',
 ' ']

In [14]:
vacancies_it_with_rs['SKILLS_SPLIT'] = vacancies_it_with_rs['REQ+SKILLS'].apply(lambda x : [word.strip() for word in re.split(r'[.;]', x)])

In [15]:
vacancies_it_with_rs.head()

,Название,Компания,Местоположение,Занятость,Требования по опыту,Зарплата,SUBCATEGORY,REQ+SKILLS,SKILLS_SPLIT
0,Программист-разработчик (SQL),ООО Финансово-правовой альянс,"Сенная площадь, Санкт-Петербург, улица Ефимова...","Полная занятость, полный день",3–6 лет,до 300 000 руб. на руки,системы управления предприятием (erp),Базы данных. Анализ данных. SQL. Разработка ПО...,"[Базы данных, Анализ данных, SQL, Разработка П..."
1,Программист C#/ .NET,ООО Syntellect (Синтеллект),Санкт-Петербург,"Полная занятость, удаленная работа",1–3 года,от 60 000 до 150 000 руб. до вычета налогов,системы управления предприятием (erp),C#. Transact-SQL. SQL. MS SQL. .NET Framework....,"[C#, Transact-SQL, SQL, MS SQL, , NET Framewor..."
3,Руководитель проекта (разработка и внедрение ПО),Наука,"Политехническая, Санкт-Петербург, Гжатская ули...","Полная занятость, полный день",3–6 лет,от 90 000 руб. на руки,системы управления предприятием (erp),MS Project. Управление проектами. Управление р...,"[MS Project, Управление проектами, Управление ..."
5,Программист 1С,Неркон,"Санкт-Петербург, Площадь Ленина, Выборгская","Полная занятость, удаленная работа",более 6 лет,от 25 000 до 45 000 руб. на руки,системы управления предприятием (erp),1С программирование. 1С: Управление Торговлей....,"[1С программирование, 1С: Управление Торговлей..."
6,Программист PHP Битрикс,ООО Вебдока,"Площадь Ленина, Санкт-Петербург, Кондратьевски...","Полная занятость, удаленная работа",3–6 лет,от 60 000 до 110 000 руб. на руки,системы управления предприятием (erp),битрикс. bitrix. yii. symfony,"[битрикс, bitrix, yii, symfony]"


# Getting full lists of key words for each vacancy

In [16]:
full_skills = vacancies_it_with_rs.groupby('SUBCATEGORY')['SKILLS_SPLIT'].sum()

In [17]:
stop_words = nltk.corpus.stopwords.words('russian') + nltk.corpus.stopwords.words('english')

In [18]:
for category in full_skills.index:
    full_skills[category] = [phrase.lower() for phrase in full_skills[category]]
    full_skills[category] = [re.sub(r'[—;–/\\,.:\+()«»]', ' ', phrase) for phrase in full_skills[category]]
    full_skills[category] = [re.sub(r'\b\d+\b', ' ', phrase) for phrase in full_skills[category]]
    full_skills[category] = [re.sub(r'-\w+', ' ', phrase) for phrase in full_skills[category]]
    full_skills[category] = [re.sub(r'\b\w\b', ' ', phrase) for phrase in full_skills[category]]
    full_skills[category] = [re.sub(r'\s+', ' ', phrase) for phrase in full_skills[category]]
    
    for stop_word in stop_words:
        full_skills[category] = [re.sub('\b' + stop_word + '\b', '', phrase) for phrase in full_skills[category]]
    
    full_skills[category] = [phrase.strip() for phrase in full_skills[category] if phrase.strip() != '' and phrase.strip() != '#'
                            and phrase.strip() != '-']

In [19]:
full_skills

SUBCATEGORY
crm системы                              [1с, битрикс24, crm, управление отношениями кл...
cto, cio, директор по it                 [apache http server, руководство коллективом, ...
web инженер                              [английский a1 начальный, django framework, gi...
web мастер                               [английский a1 начальный, django framework, gi...
администратор баз данных                 [1с программирование, обновление конфигурации ...
аналитик                                 [linux, биоинформатика опыт работы ngs данными...
банковское по                            [базы данных, анализ данных, sql, разработка п...
инженер                                  [linux, ansible, clickhouse, cassandra, git, l...
интернет                                 [работа команде, точность внимательность детал...
маркетинг                                [бизнес, английский язык, ms visio, разработка...
поддержка, helpdesk                      [apache http server, руководство колл

In [20]:
full_skills['crm системы']

['1с',
 'битрикс24',
 'crm',
 'управление отношениями клиентами',
 'автоматизация',
 'управление процессами',
 'деловая коммуникация',
 'обучение развитие',
 'ведение переговоров испытательный срок месяца',
 'оформление по тк рф',
 'зарплата формируется по итогам собеседования зависимости от вашего опыта компетенций',
 'английский b1 средний',
 'ios',
 'android',
 'flutter',
 'dart',
 'sqlite',
 'rest',
 'бухгалтерский учет',
 '1с управление торговлей',
 '1с предприятие',
 '1с комплексная автоматизация',
 'банк',
 '1с программирование',
 '1с бухгалтерия',
 'налоговая отчетность',
 'бухгалтерская отчетность',
 'основные средства',
 'тмц',
 'годовой баланс',
 'расчет ндс',
 'отчет прибылях убытках',
 'прохождение налоговых проверок',
 'реализация товаров услуг',
 'расчет затрат обязательно знание бухгалтерского учета опыт работы самостоятельной развертки настройки 1c предприятие конфигурации бух ут ка erp опыт развертки обновления конфигураций программных продуктов 1с опыт или желание об

In [49]:
itmo_data = pd.read_csv('Data/Itmo_with_keywords_30.04.2020.csv', index_col=0)

In [50]:
itmo_data.head()

,SUBFIELDCODE,SUBFIELDNAME,DEGREE,SUBJECT,SEMESTER,START_YEAR,Keywords
0,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Математика,1,2014,Элементы линейной алгебры; Математический анал...
1,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Математика,2,2014,Элементы линейной алгебры; Математический анал...
2,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Физика,1,2014,Механика; Механические колебания и волны; Элем...
3,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Физика,2,2014,Механика; Механические колебания и волны; Элем...
4,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Основы права,1,2014,


# Prepating datasets

In [51]:
itmo_data['Keywords'] = itmo_data['Keywords'].apply(lambda x: x.lower())

In [52]:
itmo_data['Keywords'] = itmo_data['Keywords'].apply(lambda x: [phrase.strip() for phrase in re.split(r'[;,]', x)])
itmo_data['Keywords'] = itmo_data['Keywords'].apply(lambda x: [re.split(r'\bи\b', phrase) for phrase in x])

In [53]:
itmo_data['Keywords'] = itmo_data['Keywords'].apply(lambda x: [word for phrase in x for word in phrase])

In [54]:
en_lemmatizer = nltk.stem.WordNetLemmatizer()
ru_lemmatizer = pymorphy2.analyzer.MorphAnalyzer()

In [55]:
itmo_data['Keywords'] = itmo_data['Keywords'].apply(lambda x: 
                                                    [' '.join([en_lemmatizer.lemmatize(word) 
                                                               if re.search(r'[a-zA-Z]+', word) 
                                                               else ru_lemmatizer.parse(word)[0].normal_form
                                                               for word in phrase.split()]) 
                                                               for phrase in x])

In [56]:
for category in full_skills.index:
    full_skills[category] = [' '.join([en_lemmatizer.lemmatize(word) 
                                                               if re.search(r'[a-zA-Z]+', word) 
                                                               else ru_lemmatizer.parse(word)[0].normal_form
                                                               for word in phrase.split()]) 
                                                               for phrase in full_skills[category]]

In [57]:
itmo_data['Keywords'] = itmo_data['Keywords'].apply(lambda x: [re.sub(r'[():/«»]', ' ', phrase) for phrase in x])

In [58]:
itmo_data['SUBJECT'] = itmo_data['SUBJECT'].apply(lambda x : re.sub(r'[():/«»]', ' ', x))

In [62]:
itmo_data

,SUBFIELDCODE,SUBFIELDNAME,DEGREE,SUBJECT,SEMESTER,START_YEAR,Keywords
0,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Математика,1,2014,"[элемент линейный алгебра, математический анал..."
1,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Математика,2,2014,"[элемент линейный алгебра, математический анал..."
2,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Физика,1,2014,"[механик, механический колебание, волна, элеме..."
3,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Физика,2,2014,"[механик, механический колебание, волна, элеме..."
4,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Основы права,1,2014,[]
...,...,...,...,...,...,...,...
53108,27.03.04,Цифровые системы управления,Академический бакалавр,Линейная алгебра Linear Algebra,1,2019,[]
53109,27.03.04,Цифровые системы управления,Академический бакалавр,Информатика Informatics,1,2019,"[стандарт оформление научный документ, графиче..."
53110,11.04.02,Создание и эксплуатация сетей и систем связи,Магистр,Методология управления проектами Предпринимат...,1,2019,[]
53111,11.04.02,Создание и эксплуатация сетей и систем связи,Магистр,Личная эффективность и управление временем,1,2019,"[ принцип, подход к оценка эффективность, м..."


In [63]:
vacancies_cat = []

for item in tqdm(itmo_data.values):
    vac_weights = []
    for name, vac in full_skills.items():
        vac_text = ' '.join(vac)    
        weight = 0
        for word in item[-1]:
            if word.strip() != '':
                if re.search(r'с\+\+', word):
                    weight += len(re.findall(r'с\+\+', vac_text))
                else:
                    weight += len(re.findall(word, vac_text))
                    
        if re.search(r'с\+\+', item[-4].lower()) or\
        item[-4].lower() == 'программирование на языках c, c++, java' or\
        re.search(r'c\+\+', item[-4].lower()):
            weight += len(re.findall(r'с\+\+', vac_text)) * 1.5
        else:
            weight += len(re.findall(item[-4].lower(), vac_text)) * 1.5
                    

        vac_weights.append((name, weight / (len(item[-1]) + len(vac))))
        
    if vac_weights != [] and vac_weights != None:
        vac_weights = sorted(vac_weights, key=lambda x: x[1], reverse=True)[0]
        if vac_weights[1] > 0:
            vacancies_cat.append(vac_weights[0])
        else:
            vacancies_cat.append(' ')


100%|██████████| 53113/53113 [17:50<00:00, 49.61it/s] 


In [65]:
itmo_data['vacancies_cat'] = vacancies_cat

In [67]:
stop_words = ['элемент', 'работа с библиотека', 'основа', 'общий концепция', 'теория', 'where to get', 'how to',
             'a', 'тема', 'общий', 'в', 'современный', 'о', 'цель', 'задача', 'содержание', 'основный',
             'раздел', 'введение в тип объект язык', 'с', 'что', 'такой', 'стандартный', 'особенность', 'язык',
             'введение', 'принцип', 'студент', 'main concept of ', 'к', 'на', 'применение', 'решение', 
             'пакет', 'они', 'возможность', 'основной', 'роль', 'more', 'on', ]

In [68]:
itmo_data['Keywords'] = itmo_data['Keywords'].apply(lambda x: [' '.join(word for word in phrase.split() if word not in stop_words) for phrase in x])

In [69]:
itmo_data.head()

,SUBFIELDCODE,SUBFIELDNAME,DEGREE,SUBJECT,SEMESTER,START_YEAR,Keywords,vacancies_cat
0,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Математика,1,2014,"[линейный алгебра, математический анализ, функ...",аналитик
1,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Математика,2,2014,"[линейный алгебра, математический анализ, функ...",аналитик
2,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Физика,1,2014,"[механик, механический колебание, волна, специ...",системы управления предприятием (erp)
3,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Физика,2,2014,"[механик, механический колебание, волна, специ...",системы управления предприятием (erp)
4,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Основы права,1,2014,[],


In [70]:
documents_hh = [TaggedDocument((' '.join(doc)).split(' '), [name]) for name,  doc in full_skills.items() if doc != []]
documents_itmo = [TaggedDocument((' '.join(doc)).split(' '), [' '.join((' '.join(doc)).split(' '))]) for doc in np.unique(itmo_data['Keywords']) if doc != []]

In [71]:
# constructing generic dataset
documents = documents_hh + documents_itmo

In [72]:
model = Doc2Vec(documents, dm=0, alpha=0.025, size=20, min_alpha=0.025, min_count=0)

/home/blakexdd/Documents/Github/Matching/venv/lib/python3.8/site-packages/gensim/models/doc2vec.py:574: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [73]:
for epoch in tqdm(range(100)):
    model.train(documents, total_examples=model.corpus_count, epochs=10)
    model.alpha -= 0.002
    model.min_alpha = model.alpha

100%|██████████| 100/100 [01:05<00:00,  1.53it/s]


In [74]:
itmo_data.head()

,SUBFIELDCODE,SUBFIELDNAME,DEGREE,SUBJECT,SEMESTER,START_YEAR,Keywords,vacancies_cat
0,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Математика,1,2014,"[линейный алгебра, математический анализ, функ...",аналитик
1,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Математика,2,2014,"[линейный алгебра, математический анализ, функ...",аналитик
2,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Физика,1,2014,"[механик, механический колебание, волна, специ...",системы управления предприятием (erp)
3,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Физика,2,2014,"[механик, механический колебание, волна, специ...",системы управления предприятием (erp)
4,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Основы права,1,2014,[],


In [82]:
vac = []

for desc in tqdm(itmo_data.values):
    most_sim = 0
    vacn = ' '
    
    if desc[-1] == ' ':
        desc_sim = []
        for name, vacancy in full_skills.items():
            sim = model.docvecs.similarity(' '.join(desc[-2]), name)
            desc_sim.append((desc[3], sim, name))

        vac.append(sorted(desc_sim, key=lambda x:x[1], reverse=True)[0][-1])
    else:
        vac.append(' ')

    

100%|██████████| 53113/53113 [00:05<00:00, 9972.10it/s] 


In [84]:
itmo_data['vacancies_cat'] = itmo_data['vacancies_cat'] + vac

In [94]:
itmo_data[:50]

,SUBFIELDCODE,SUBFIELDNAME,DEGREE,SUBJECT,SEMESTER,START_YEAR,Keywords,vacancies_cat
0,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Математика,1,2014,"[линейный алгебра, математический анализ, функ...",аналитик
1,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Математика,2,2014,"[линейный алгебра, математический анализ, функ...",аналитик
2,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Физика,1,2014,"[механик, механический колебание, волна, специ...",системы управления предприятием (erp)
3,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Физика,2,2014,"[механик, механический колебание, волна, специ...",системы управления предприятием (erp)
4,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Основы права,1,2014,[],инженер
5,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Основы программирования,1,2014,"[программирование, парадигма программирование,...","cto, cio, директор по it"
6,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Основы программирования,2,2014,"[программирование, парадигма программирование,...","cto, cio, директор по it"
7,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Основы вычислительной техники,1,2014,"[, построение эвм, архитектура персональный ко...",интернет
8,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Основы вычислительной техники,2,2014,"[, построение эвм, архитектура персональный ко...",интернет
9,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Физическая культура,1,2014,"[здоровый образ жизнь, спорт, физиологический,...","cto, cio, директор по it"


In [95]:
itmo_data.to_csv('Data/itmo_with_vacancies_11_may_2020.csv')

In [92]:
itmo_data['SUBJECT'].unique().shape

(6928,)